In [2]:
import arcpy
from arcgis.gis import GIS

import numpy as np
import matplotlib.pyplot as plt
import seaborn

import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor

gis = GIS()
arcpy.env.overwriteOutput = True

In [3]:
arcpy.env.workspace = r"C:\Users\drewt\Desktop\Stuff\college\GEOG4324\Palm_Tree_Detection\Kolovai"

In [3]:
with arcpy.EnvManager(cellSize="Imagery"):
    arcpy.ia.ExportTrainingDataForDeepLearning(
        in_raster="Imagery",
        out_folder=r"C:\Users\drewt\Desktop\Stuff\college\GEOG4324\Palm_Tree_Detection\Kolovai\imagechips",
        in_class_data=r"C:\Users\drewt\Desktop\Stuff\college\GEOG4324\Palm_Tree_Detection\Kolovai\Kolovai.gdb\PalmTraining",
        image_chip_format="JPEG",
        tile_size_x=448,
        tile_size_y=448,
        stride_x=128,
        stride_y=128,
        output_nofeature_tiles="ONLY_TILES_WITH_FEATURES",
        metadata_format="PASCAL_VOC_rectangles",
        start_index=0,
        class_value_field="Classvalue",
        buffer_radius=0,
        in_mask_polygons=None,
        rotation_angle=0,
        reference_system="MAP_SPACE",
        processing_mode="PROCESS_AS_MOSAICKED_IMAGE",
        blacken_around_feature="NO_BLACKEN",
        crop_mode="FIXED_SIZE",
        in_raster2=None,
        in_instance_data=None,
        instance_class_value_field=None,
        min_polygon_overlap_ratio=0
    )

In [1]:
with arcpy.EnvManager(processorType="GPU"):
    arcpy.ia.TrainDeepLearningModel(
        in_folder=r"C:\Users\drewt\Desktop\Stuff\college\GEOG4324\Palm_Tree_Detection\Kolovai\imagechips",
        out_folder=r"C:\Users\drewt\Desktop\Stuff\college\GEOG4324\Palm_Tree_Detection\Kolovai\classify_palms",
        max_epochs=50,
        model_type="SSD",
        batch_size=8,
        arguments="grids #;zooms 1.0;ratios '[1.0, 1.0]';chip_size 224;resize_to #;monitor valid_loss",
        learning_rate=None,
        backbone_model="RESNET34",
        pretrained_model=None,
        validation_percentage=10,
        stop_training="CONTINUE_TRAINING",
        freeze="FREEZE_MODEL"
    )

epoch     train_loss  valid_loss  average_precision  time    
0         56.685307   58.941448   0.244435           01:39     
1         52.618252   55.362293   0.304323           01:31     
2         48.798470   55.017441   0.337629           01:32     
3         49.444073   52.383301   0.365801           01:33     
4         46.499485   50.763569   0.402648           01:32     
5         44.229313   51.975948   0.351285           01:32     
6         45.500118   50.549446   0.415116           01:32     
7         42.824081   48.165230   0.448628           01:32     
8         41.549381   48.827797   0.444910           01:32     
9         39.976288   47.378819   0.426998           01:32     
10        41.181046   46.194977   0.444613           01:32     
11        36.857441   45.513466   0.473475           01:32     
12        38.489326   45.277859   0.517168           01:32     
13        37.072277   45.016834   0.490803           01:32     
14        38.029457   43.713203   0.488488

In [1]:
# ArcGIS crashed when computing the model metrics, so they will not show up on my notebook.

In [4]:
with arcpy.EnvManager(extent='-19519402.0714087 -2403701.11028656 -19519199.3472533 -2403564.13450585 PROJCS["unnamed_ellipse_Mercator_2SP",GEOGCS["GCS_unnamed_ellipse",DATUM["D_unknown",SPHEROID["Unknown",6378137.0,0.0]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_2SP"],PARAMETER["false_easting",0.0],PARAMETER["false_northing",0.0],PARAMETER["central_meridian",0.0],PARAMETER["standard_parallel_1",0.0],UNIT["Meter",1.0]]', cellSize="Imagery", processorType="GPU"):
    out_classified_raster = arcpy.ia.DetectObjectsUsingDeepLearning(
        in_raster="Imagery",
        out_detected_objects=r"C:\Users\drewt\Desktop\Stuff\college\GEOG4324\Palm_Tree_Detection\Kolovai\Kolovai.gdb\DetectedPalms",
        in_model_definition=r"C:\Users\drewt\Desktop\Stuff\college\GEOG4324\Palm_Tree_Detection\Kolovai\classify_palms\classify_palms.dlpk",
        arguments="padding 56;threshold 0.2;nms_overlap 0.1;batch_size 8;exclude_pad_detections True;test_time_augmentation False",
        run_nms="NO_NMS",
        confidence_score_field="Confidence",
        class_value_field="Class",
        max_overlap_ratio=0,
        processing_mode="PROCESS_AS_MOSAICKED_IMAGE"
    )
out_classified_raster.save(None)

AttributeError: 'ArcGISObjectDetector' object has no attribute 'updatePixels'


AttributeError: ResultObject: Get attribute save not found

In [5]:
with arcpy.EnvManager(extent="DEFAULT", cellSize="Imagery", processorType="GPU"):
    out_classified_raster = arcpy.ia.DetectObjectsUsingDeepLearning(
        in_raster="Imagery",
        out_detected_objects=r"C:\Users\drewt\Desktop\Stuff\college\GEOG4324\Palm_Tree_Detection\Kolovai\Kolovai.gdb\DetectedPalms",
        in_model_definition=r"C:\Users\drewt\Desktop\Stuff\college\GEOG4324\Palm_Tree_Detection\Kolovai\classify_palms\classify_palms.dlpk",
        arguments="padding 56;threshold 0.5;nms_overlap 0.01;batch_size 8;exclude_pad_detections True;test_time_augmentation False",
        run_nms="NO_NMS",
        confidence_score_field="Confidence",
        class_value_field="Class",
        max_overlap_ratio=0,
        processing_mode="PROCESS_AS_MOSAICKED_IMAGE"
    )
out_classified_raster.save(None)

AttributeError: 'ArcGISObjectDetector' object has no attribute 'updatePixels'


AttributeError: ResultObject: Get attribute save not found

In [6]:
arcpy.ia.NonMaximumSuppression(
    in_featureclass="DetectedPalms",
    confidence_score_field="Confidence",
    out_featureclass=r"C:\Users\drewt\Desktop\Stuff\college\GEOG4324\Palm_Tree_Detection\Kolovai\Kolovai.gdb\DetectedPalms_NMS",
    class_value_field="Class",
    max_overlap_ratio=0.5
)

<geoprocessing server result object object at 0x000001BB1DDF2F60>

In [7]:
arcpy.management.FeatureToPoint(
    in_features="Detected Palm Trees",
    out_feature_class=r"C:\Users\drewt\Desktop\Stuff\college\GEOG4324\Palm_Tree_Detection\Kolovai\Kolovai.gdb\PalmTree_Points",
    point_location="CENTROID"
)

<Result 'C:\\Users\\drewt\\Desktop\\Stuff\\college\\GEOG4324\\Palm_Tree_Detection\\Kolovai\\Kolovai.gdb\\PalmTree_Points'>

In [8]:
arcpy.analysis.PairwiseBuffer(
    in_features="PalmTree_Points",
    out_feature_class=r"C:\Users\drewt\Desktop\Stuff\college\GEOG4324\Palm_Tree_Detection\Kolovai\Kolovai.gdb\PalmTreeBuffer",
    buffer_distance_or_field="3 Meters",
    dissolve_option="NONE",
    dissolve_field=None,
    method="PLANAR",
    max_deviation="0 Meters"
)

<Result 'C:\\Users\\drewt\\Desktop\\Stuff\\college\\GEOG4324\\Palm_Tree_Detection\\Kolovai\\Kolovai.gdb\\PalmTreeBuffer'>

In [9]:
arcpy.ia.ZonalStatisticsAsTable(
    in_zone_data="PalmTreeBuffer",
    zone_field="ORIG_FID",
    in_value_raster="VARI",
    out_table=r"C:\Users\drewt\Desktop\Stuff\college\GEOG4324\Palm_Tree_Detection\Kolovai\Kolovai.gdb\MeanVARI_per_Palm",
    ignore_nodata="DATA",
    statistics_type="MEAN",
    process_as_multidimensional="CURRENT_SLICE",
    percentile_values=90,
    percentile_interpolation_type="AUTO_DETECT",
    circular_calculation="ARITHMETIC",
    circular_wrap_value=360
)

<geoprocessing server result object object at 0x000001BB2BC48840>

In [10]:
arcpy.management.JoinField(
    in_data="PalmTreeBuffer",
    in_field="ORIG_FID",
    join_table="MeanVARI_per_Palm",
    join_field="ORIG_FID",
    fields="MEAN",
    fm_option="NOT_USE_FM",
    field_mapping=None
)

<Result 'PalmTreeBuffer'>